## САМОСТОЯТЕЛЬНАЯ РАБОТА
## Исследование поведения пользователей


#### Необходимо проверить два предположения:
 - Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
 - Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

Проверку произвести на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

#### План:
 - Получить данные из файла.

 - Произвести обзор данных и преобразование данных, если оно необходимо.

 - Выделить группы пользователей по событиям выбора уровня сложности (чтобы найти зависимость между фактом выбора уровня      сложности и оплатой).

 - Рассчитать для каждой группы процент оплат.

 - Для каждой группы сделать по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты.

 - Объединить датафреймы в рамках одной группы и найти разницу во времени между событиями.

 - Рассчитайте среднее время между событиями выбора уровня сложности и оплатой.
 ###### Дополнительно:
 - Расcчитать количество и долю пользователей, которые приступили к этапу выбора уровня сложности.
 - Рассчитать процент выбора пользователями различных уровней сложности.
 - Рассчитать среднее время между событиями регистрации и оплатой у групп пользователей, выбравших разные уровни сложности.
 
- СДЕЛАТЬ ВЫВОДЫ

#### Получение данных из файла 

In [1]:
import pandas as pd          
df= pd.read_csv('Events.csv')
df.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [2]:
import pandas as pd  
pdf=pd.read_csv('Purchase.xls')
pdf.head()

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


#### Обзор и преобразование данных.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [4]:
df.describe(include='all')

,id,event_type,selected_level,start_time,tutorial_id,user_id
count,252334.00000,252334,31086,252334,125103.000000,252334.000000
unique,NaN,5,3,251565,NaN,NaN
top,NaN,registration,medium,2017-05-29T11:03:08,NaN,NaN
freq,NaN,74908,17408,16,NaN,NaN
mean,155069.50000,NaN,NaN,NaN,51815.675364,50003.098790
std,72842.69575,NaN,NaN,NaN,19792.496704,21586.690775
min,28903.00000,NaN,NaN,NaN,17562.000000,12583.000000
25%,91986.25000,NaN,NaN,NaN,34666.000000,31302.000000
50%,155069.50000,NaN,NaN,NaN,51794.000000,49983.000000
75%,218152.75000,NaN,NaN,NaN,68961.500000,68670.000000


In [5]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [6]:
pdf.describe()

,id,user_id,amount
count,5956.000000,5956.000000,5956.000000
mean,18651.500000,50224.943251,108.919577
std,1719.493433,21458.351127,55.786902
min,15674.000000,12584.000000,25.000000
25%,17162.750000,31701.500000,50.000000
50%,18651.500000,50343.500000,100.000000
75%,20140.250000,68584.500000,150.000000
max,21629.000000,87480.000000,350.000000


In [7]:
reg_df_2018=(df.start_time>='2018-01-01')&(df.start_time<'2019-01-01')&(df.event_type=='registration')
#Выбираем из данных только тех, кто зарегистрировался в 2018 году

registered=df[reg_df_2018]['user_id'].to_list()
# Создаем список пользователей, зарегистрировннах в 2018 году

events_2018=df[df.user_id.isin(registered)]
### Создаем датафрейм с пользователями, которые зарегистрировались в 2018 году

purchase_2018=pdf[pdf.user_id.isin(registered)]
### Создаем датафрейм с данными об оплатах, которые совершают пользователи, зарегистрированные в 2018 году

In [8]:
events_2018.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01T03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01T04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01T08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01T11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01T13:28:07,NaN,27836


In [9]:
events_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


In [10]:
purchase_2018.head()

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03T18:53:43,100
1172,16846,27865,2018-01-04T14:46:10,250
1174,16848,27911,2018-01-07T08:19:12,50
1175,16849,27910,2018-01-07T12:11:34,100
1176,16850,27940,2018-01-07T13:16:41,200


In [11]:
purchase_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


In [12]:
# Переименуем столбцы

events_2018=events_2018.rename(columns={'id':'event_id'})
purchase_2018=purchase_2018.rename(columns={'id':'purchase_id'})

In [13]:
# Преобразуем данные строкового типа в datetime

events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])

In [14]:
# Добавим в датафрейм purchase_2018 столбец event_type, который будет содержать одно значение purchase. 
# Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

purchase_2018['event_type'] = 'purchase'
purchase_2018.head()

,purchase_id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03 18:53:43,100,purchase
1172,16846,27865,2018-01-04 14:46:10,250,purchase
1174,16848,27911,2018-01-07 08:19:12,50,purchase
1175,16849,27910,2018-01-07 12:11:34,100,purchase
1176,16850,27940,2018-01-07 13:16:41,200,purchase


In [15]:
purchase_2018.describe()

,purchase_id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


In [17]:
### Оценим, какие уникальные события есть в колонке selected_level:
events_2018['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [18]:
# Объединим датафреймы events_2018 и purchase_2018 с помощью функции pd.concat() и запишем объединенный датафрейм 
# в переменную total_events.
total_events=pd.concat([events_2018, purchase_2018])
total_events

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2767,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
2768,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
2769,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
2774,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


In [19]:
total_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [20]:
# Cбросим индексы объединенного датафрейма (так как после объединения они дублировались и несут мало смысла)
# с помощью метода reset_index() и отсортируем все события по возрастанию времени с помощью sort_values():
total_events=total_events.reset_index(drop=True).sort_values('start_time')
total_events

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
68555,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
68556,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
68557,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


In [21]:
# Количество уникальных пользователей, зарегистрировавшихся в 2018 году
user_total_events=total_events['user_id'].nunique() ## количество уникальных пользователей, зарегистрировавшихся в 2018 году
user_total_events

19926

#### Расчет количества и доли пользователей, которые приступили к этапу выбора уровня сложности.


In [22]:
# Количество пользователей, которые приступили к выбору уровня сложности 
level_choice=total_events[total_events['event_type']=='level_choice']
user_level_choice=level_choice['user_id'].nunique()
user_level_choice

8342

In [23]:
#Процент пользователей, которые приступили к выбору уровня сложности от общего количества пользователей
persent_user_level_choice=user_level_choice*100/user_total_events
persent_user_level_choice

41.86490013048279

#### Расчет количества и доли пользователей, выбравших разные уровни сложности (easy, medium, hard)

In [24]:
# EASY

#Кол-во уникальных пользователей, которые выбрали легкий (easy) уровень сложности
level_choice_easy=level_choice[level_choice['selected_level']=='easy']
user_level_choice_easy=level_choice_easy['user_id'].nunique()
user_level_choice_easy 

2448

In [25]:
#Процент уникальных пользователей, которые выбрали легкий (easy) уровня сложности
persent_user_level_choice_easy=user_level_choice_easy*100/user_level_choice
persent_user_level_choice_easy 

29.345480700071924

In [26]:
# MEDIUM

#Кол-во уникальных пользователей, которые выбрали средний (medium) уровень сложности
level_choice_medium=level_choice[level_choice['selected_level']=='medium']
user_level_choice_medium=level_choice_medium['user_id'].nunique()
user_level_choice_medium

4645

In [27]:
# Процент уникальных пользователей, которые выбрали средний (medium) уровень сложности
persent_user_level_choice_medium=user_level_choice_medium*100/user_level_choice
persent_user_level_choice_medium 

55.68209062574922

In [28]:
# HARD

# Кол-во уникальных пользователей, которые выбрали тяжелый (hard) уровень сложности
level_choice_hard=level_choice[level_choice['selected_level']=='hard']
user_level_choice_hard=level_choice_hard['user_id'].nunique()
user_level_choice_hard 

1249

In [29]:
# Процент уникальных пользователей, которые выбрали тяжелый (hard) уровень сложности
persent_user_level_choice_hard=user_level_choice_hard*100/user_level_choice
persent_user_level_choice_hard 

14.972428674178854

#### Выделяем группы пользователей по выбору уровня сложности и оплате



In [30]:
# EASY
level_choice_easy = total_events[total_events['selected_level'] == 'easy']['user_id'].unique() 
# список пользоват., которые выбрали легкий уровень

purchase_2018_easy=purchase_2018[purchase_2018['user_id'].isin(level_choice_easy)]
# пользователи, которые выбрали легк.уровень и совершили оплату


# MEDIUM
level_choice_medium=total_events[total_events['selected_level']=='medium']['user_id'].unique() 
#список пользоват., которые выбрали средний уровень

purchase_2018_medium=purchase_2018[purchase_2018['user_id'].isin(level_choice_medium)]
# пользователи, которые выбрали средн.уровень и совершили оплату

# HARD
level_choice_hard=total_events[total_events['selected_level']=='hard']['user_id'].unique()
# список пользоват., которые выбрали тяжелый уровень

purchase_2018_hard=purchase_2018[purchase_2018['user_id'].isin(level_choice_hard)]#выбираем оплаты только тех 
# пользователи, которые выбрали тяжел.уровень и совершили оплату

#### Расчет процента пользователей, выбравших разные уровни сложности, которые совершили оплату

In [31]:
#EASY
persent_purchase_easy=purchase_2018_easy['user_id'].nunique()/len(level_choice_easy)

#MEDIUM
persent_purchase_medium=purchase_2018_medium['user_id'].nunique()/len(level_choice_medium)

##HARD
persent_purchase_hard=purchase_2018_hard['user_id'].nunique()/len(level_choice_hard)

print('Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших easy_level: {:.2%}'.format (persent_purchase_easy))
print('Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших medium_level: {:.2%}'.format (persent_purchase_medium))
print('Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших hard_level: {:.2%}'.format (persent_purchase_hard))

Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших easy_level: 7.72%
Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших medium_level: 20.86%
Процент пользователей, совершивих оплату, от общего количества пользователей, выбравших hard_level: 35.39%


#### Расчет разницы во времени между событиями выбора уровня сложности и оплаты для разных групп пользователей 

Для каждой группы сделаем по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты

In [32]:
# Создадим датафреймы с событиями выбора уровня сложности и оставит только необходимые столбцы

#EASY
easy=total_events[(total_events['event_type']=='level_choice')& (total_events['selected_level']=='easy')]
easy=easy[['user_id','start_time']].rename(columns={'start_time':'easy_time'})
easy.head()

,user_id,easy_time
45,27842,2018-01-02 08:46:03
52,27849,2018-01-02 11:53:11
88,27853,2018-01-03 00:39:54
92,27859,2018-01-03 01:11:45
108,27863,2018-01-03 09:31:00


In [33]:
#MEDIUM
medium=total_events[(total_events['event_type']=='level_choice')&(total_events['selected_level']=='medium')]
medium=medium[['user_id','start_time']].rename(columns={'start_time':'medium_time'})
medium.head()

,user_id,medium_time
19,27835,2018-01-01 20:37:22
36,27840,2018-01-02 05:18:42
57,27843,2018-01-02 14:09:58
59,27846,2018-01-02 15:10:27
68,27847,2018-01-02 18:09:02


In [34]:
#HARD
hard=total_events[(total_events['event_type']=='level_choice')&(total_events['selected_level']=='hard')]
hard=hard[['user_id','start_time']].rename(columns={'start_time':'hard_time'})
hard.head()

,user_id,hard_time
23,27839,2018-01-01 22:37:50
38,27845,2018-01-02 06:19:18
151,27865,2018-01-04 05:56:32
201,27888,2018-01-04 19:41:29
237,27910,2018-01-05 11:59:50


In [35]:
# Создадим датафрейм с событиями оплаты, в котором оставим только нужные столбцы
purchase=purchase_2018[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
purchase.head()

,user_id,purchase_time
1171,27845,2018-01-03 18:53:43
1172,27865,2018-01-04 14:46:10
1174,27911,2018-01-07 08:19:12
1175,27910,2018-01-07 12:11:34
1176,27940,2018-01-07 13:16:41


In [36]:
# Проверим, что в нашем датафрейме purchase для каждого пользователя содержится только по одному событию
purchase['user_id'].value_counts().mean()

1.0

In [37]:
# Объединим датафрейм с событием и оплатой для каждой группы и посчитаем среднюю разницу во времени между выбором уровня 
# сложности и оплатой. 

#EASY
merged_easy=easy.merge(purchase, on='user_id', how='inner')
merged_easy['timedelta']=merged_easy['purchase_time']-merged_easy['easy_time']
merged_easy['timedelta'].mean()
merged_easy['timedelta'].describe()

count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object

In [38]:
#MEDIUM
merged_medium=medium.merge(purchase, on='user_id', how='inner')
merged_medium['timedelta']=merged_medium['purchase_time']-merged_medium['medium_time']
merged_medium['timedelta'].mean()
merged_medium['timedelta'].describe()

count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object

In [39]:
#HARD
merged_hard=hard.merge(purchase, on='user_id', how='inner')
merged_hard['timedelta']=merged_hard['purchase_time']-merged_hard['hard_time']
merged_hard['timedelta'].mean()
merged_hard['timedelta'].describe()

count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object

In [40]:
print('Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших easy_level:{}'.format (merged_easy['timedelta'].mean()))
print('Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших medium_level:{}'.format (merged_medium['timedelta'].mean()))
print('Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших hard_level:{}'.format (merged_hard['timedelta'].mean()))

Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших easy_level:3 days 14:58:52.941798941
Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших medium_level:3 days 23:14:13.165118679
Среднее время между событиями выбора уровня сложности и оплаты для группы пользователей, выбравших hard_level:3 days 07:20:41.420814479


#### Расчет разницы во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности

In [41]:
# Выделим отдельный датафрейм registration_2018, который будет содержать только события с event_type = registration. 
# Этот датафрейм будет вспомогательным для определения времени между регистрацией и оплатой.
# Оставляем только те столбцы, которые нам нужны
registration_2018 = total_events[total_events['event_type'] == 'registration']
registration_2018=registration_2018[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
registration_2018.head()

,user_id,registration_time
0,27832,2018-01-01 03:48:40
1,27833,2018-01-01 04:07:25
2,27834,2018-01-01 08:35:10
3,27835,2018-01-01 11:54:47
4,27836,2018-01-01 13:28:07


In [42]:
# Объединим, созданные ранее датафреймы с событиями и оплатой с датафреймом registration_2018 и посчитаем среднее время 
# между событиями регистрации и оплаты для группы пользователей, выбравших разные уровни сложности

#EASY
merged_easy_reg=registration_2018.merge(merged_easy, on='user_id', how='inner')
merged_easy_reg['timedelta_2']=merged_easy_reg['purchase_time']-merged_easy_reg['registration_time']
merged_easy_reg['timedelta_2'].mean()
merged_easy_reg['timedelta_2'].describe()

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta_2, dtype: object

In [43]:
# MEDIUM
merged_medium_reg=registration_2018.merge(merged_medium, on='user_id', how='inner')
merged_medium_reg['timedelta2']=merged_medium_reg['purchase_time']-merged_medium_reg['registration_time']
merged_medium_reg['timedelta2'].mean()
merged_medium_reg['timedelta2'].describe()

count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta2, dtype: object

In [44]:
#HARD
merged_hard_reg=registration_2018.merge(merged_hard, on='user_id', how='inner')
merged_hard_reg['timedelta2']=merged_hard_reg['purchase_time']-merged_hard_reg['registration_time']
merged_hard_reg['timedelta2'].mean()
merged_hard_reg['timedelta2'].describe()

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta2, dtype: object

In [45]:
print('Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших easy_level:{}'.format (merged_easy_reg['timedelta_2'].mean()))
print('Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших medium_level:{}'.format (merged_medium_reg['timedelta2'].mean()))
print('Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших hard_level:{}'.format (merged_hard_reg['timedelta2'].mean()))

Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших easy_level:3 days 22:10:23.211640211
Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших medium_level:4 days 06:12:06.576883384
Среднее время между событиями регистрации и оплаты для группы пользователей, выбравших hard_level:3 days 14:55:19.257918552


### ВЫВОДЫ:

#### 1. До этапа выбора уровня сложности доходит всего 41,86%. Наблюдается прямая зависимость между выбором уровня сложности и оплатой. Чем выше уровень сложности, который выбрал пользователь, тем вероятнее оплата. Однако, стоит обратить внимание на то, что 55,68% выбирают средний уровень и всего 14,97% - тяжелый уровень.¶
 - Доля пользователей, которые приступили к выбору уровня сложности от общего количества пользователей - 41,86%
 
 - Доля пользователей, выбравших разные уровни сложности:
   - easy_level - 29.35%
   - medium_level - 55.68% 
   - hard_level - 14.97%

 - Процент пользователей, выбравших разные уровни сложности, которые совершили ОПЛАТУ
   - easy_level - 7.72%
   - medium_level - 20.86% 
   - hard_level - 35.39%
   
#### 2. Самая большая разница  во времени между событиями регистрации и оплаты наблюдается у группы пользователей, выбравших средний уровень сложности. Быстрее всего выбирают платный пакет пользователи, выбравшие сложный уровень. А вот среднее время между выбором уровня сложности и оплатой у разных групп отличается незначительно.
  
 
- Среднее время между событиями регистрации и оплаты для групп пользователей, выбравших разные уровни сложности: 
   - easy_level - 3 дня 22 часа 10 минут
   - medium_level - 4 дня 06 часов 12 минут
   - hard_level - 3 дня 14 часов 55 минут

- Среднее время между событиями выбора уровня сложности и оплаты для групп пользователей, выбравших разные уровни сложности: 
   - easy_level - 3 дня 14 часов 58 минут
   - medium_level - 3 дня 23 часа 14 минут
   - hard_level - 3 дня 7 часов 20 минут
   
#### Самая результативная группа - пользователи, выбравшие сложный (hard) уровень, но они составляют всего 14,97% процентов. 
